Build HMM profiles for plastic-degrading enzymes.

Usage:
```bash
python build_hmms.py --fasta enzymes.faa --outdir hmms \
                        --cdhit_id 0.95 --min_seqs 4
```

In [2]:
# Libraries
import argparse, subprocess, tempfile, shutil, os, json
from pathlib import Path
from typing import List, Dict
from Bio import SeqIO, SearchIO
import requests, time
from dotenv import load_dotenv


In [ ]:
# Configuration
## Working Directory
load_dotenv()
working_directory = os.getenv("working_dir")
os.chdir(working_directory)
print(os.getcwd())
## Paths
### Databases
if os.path.exists("input/uniprot_sprot.fasta"):
    print("Using local uniprot_sprot.fasta")
    Local_Uniprot_Database = "input/uniprot_sprot.fasta"
else:
    print("Missing local uniprot_sprot.fasta. \n To download it (47GB), run:\n wget https://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref90/uniref90.fasta.gz")

Plastic_Seq_Database = "input/TEST_FASTA.fasta" #"input/PlasticDB.fasta"

### Outputs
First_Cluster_output_path = "output/First_Cluster/"
Second_Cluster_output_path = "output/Second_Cluster/"

## Parameters
CDHIT_Identity = 0.95
CDHIT_prefix = "CDHIT"
WORD_SIZE = 5

/home/marcleo/work/git_reps/PlasticDB-HMM
Missing local uniprot_sprot.fasta. 
 To download it (47GB), run:
 wget https://ftp.uniprot.org/pub/databases/uniprot/uniref/uniref90/uniref90.fasta.gz


In [ ]:
# Step 1: Cluster sequences using CD-HIT

def run_cdhit(input_fasta, output_path, output_prefix, identity, word_size):
    output_file = f"{output_path}{output_prefix}.fasta"
    mdhit_cmd = [
        "cd-hit",
        "-i", input_fasta,
        "-o", output_file,
        "-c", str(identity),
        "-n", str(word_size),
        "-g", "1",
    ]
    subprocess.run(mdhit_cmd, check=True)
    print(f"CD-HIT clustering completed. Output: {output_prefix}.fasta")
    return output_file

run_cdhit(Plastic_Seq_Database, First_Cluster_output_path, CDHIT_prefix, CDHIT_Identity, WORD_SIZE)

Program: CD-HIT, V4.8.1 (+OpenMP), Apr 24 2025, 22:00:32
Command: cd-hit -i input/TEST_FASTA.fasta -o
         output/First_Cluster/CDHIT.fasta -c 0.95 -n 5 -g 1

Started: Wed May 21 19:51:37 2025
                            Output                              
----------------------------------------------------------------
total seq: 45
longest and shortest : 639 and 201
Total letters: 20446
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 16M = 16M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 81M

Table limit with the given memory limit:
Max number of representatives: 1208273
Max number of word counting entries: 89810798

comparing sequences from          0  to         45

       45  finished         37  clusters

Approximated maximum memory consumption: 81M
writing new database
writing clustering information
program completed !

Total CPU time 0.05
CD-HIT clustering completed. Output: CDHIT.fas

'output/First_Cluster/CDHIT.fasta'

In [ ]:
# Step 1.2: Parse CD-HIT output to get representative sequences per cluster
def parse_cdhit_output(output_path):
    cdht_metadata_file = f"{output_path}{CDHIT_prefix}.fasta.clstr"
    clusters = {}
    with open(cdht_metadata_file, "r") as a:
        for line in a:
            if line.startswith(">Cluster"):
                cluster_id = line.split()[1]
                clusters[cluster_id] = []
            if line.endswith("*\n"):
                seq_id = line.split()[2]
                seq_id = seq_id[0:6]
                clusters[cluster_id].append(seq_id)
    return clusters

representative_sequences = parse_cdhit_output(First_Cluster_output_path)
print(representative_sequences)

{'0': ['>00010'], '1': ['>00012'], '2': ['>00016'], '3': ['>00028'], '4': ['>00039'], '5': ['>00026'], '6': ['>00018'], '7': ['>00006'], '8': ['>00023'], '9': ['>00032'], '10': ['>00013'], '11': ['>00017'], '12': ['>00025'], '13': ['>00011'], '14': ['>00001'], '15': ['>00029'], '16': ['>00014'], '17': ['>00019'], '18': ['>00035'], '19': ['>00003'], '20': ['>00005'], '21': ['>00049'], '22': ['>00007'], '23': ['>00004'], '24': ['>00043'], '25': ['>00044'], '26': ['>00022'], '27': ['>00033'], '28': ['>00048'], '29': ['>00042'], '30': ['>00024'], '31': ['>00047'], '32': ['>00002'], '33': ['>00036'], '34': ['>00037'], '35': ['>00008'], '36': ['>00027']}


In [53]:
# Step 1.3: Write representative sequences in a new FASTA file

for seq_id in representative_sequences:
    seqs = representative_sequences[seq_id]
    for seq in seqs:
        with open (Plastic_Seq_Database, "r") as reference_fasta:
            reference_lines = reference_fasta.readlines()
            for line in range(len(reference_lines)):
                if reference_lines[line].startswith(seq):
                    with open(f"{First_Cluster_output_path}{CDHIT_prefix}_representative.fasta", "a") as output_fasta:
                        if line + 1 < len(reference_lines) and not reference_lines[line + 1].startswith(">"):
                            output_fasta.write(reference_lines[line])
                            output_fasta.write(reference_lines[line + 1])

